#  Data Wrangling and Analyzing on "We Rate Dogs" Twitter Site
by Dilupa Chandrasekara

## Data Gathering

1. Twitter Archive File : Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)
2. The tweet image predictions - what breed of dog (or other things) is present in each tweet according to a neural network(Previously done and tested). This file (image_predictions.tsv) is hosted on Udacity's servers and needed to be downloaded programmatically using the Requests library and the following URL -https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

3. Twitter API & JSON - Each tweet's found in the We rate dogs we retweet count and favorite ("like") count at minimum. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count.

1. Twitter Archive File

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests 
import seaborn as sns
import time



In [2]:
#Read CSV file 
df = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

2. Uses the Requests library to download the tweet image prediction (image_predictions.tsv)

In [5]:
import requests
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
data = requests.get(url)
with open (url.split('/')[-1], mode='wb') as file:
    file.write(data.content)

In [6]:
image_data = pd.read_csv('image-predictions.tsv', sep='\t')
image_data.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [7]:
#!pip install  --upgrade tweepy==4.6

3. Uses the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [ ]:
import tweepy

bearer_token='AAAAAAAAAAAAAAAAAAAAAMPpYwEAAAAAHYmhw4BflEGucKBU4ZBlQFjMVn0%3DMMIPovZjtqjof7qGLBpNZAOus8JxB6wky7w4Hba08CMIOQw2Nu'
# create instance
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

# fields to return in query
tweet_fields=["id","in_reply_to_user_id","favourite_count", "retweet_count", "retweet_status","lang",
    "public_metrics","source","text"]
# query

tweet_id_list = df['tweet_id']

tweet_field = ["id","in_reply_to_user_id","lang",
    "public_metrics","source","text"]
public_metrics_list = []

for tweet_id in tweet_id_list:
        tweet_dict={}
        temp = client.get_tweet(id=tweet_id, tweet_fields =tweet_field)
        tweet_data = temp.data
        for key in tweet_field:
            if tweet_data and tweet_data.get(key):
                tweet_dict[key]=tweet_data[key]
            else:
                print(f'Empty tweet: {tweet_id}, tweet info: {tweet_data}')
            
        public_metrics_list.append(tweet_dict)
        

Empty tweet: 892420643555336193, tweet info: This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU
Empty tweet: 892177421306343426, tweet info: This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV
Empty tweet: 891815181378084864, tweet info: This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB
Empty tweet: 891689557279858688, tweet info: This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ
Empty tweet: 891327558926688256, tweet info: This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f
Empty tweet: 891087950875897856, tweet info: Here we have a majestic great white breachin

Empty tweet: 883117836046086144, tweet info: Please only send dogs. We don't rate mechanics, no matter how h*ckin good. Thank you... 13/10 would sneak a pat https://t.co/Se5fZ9wp5E
Empty tweet: 882992080364220416, tweet info: This is Rusty. He wasn't ready for the first pic. Clearly puppared for the second. 13/10 confirmed great boy https://t.co/tyER0KpdXj
Empty tweet: 882762694511734784, tweet info: This is Gus. He's quite the cheeky pupper. Already perfected the disinterested wink. 12/10 would let steal my girl https://t.co/D43I96SlVu
Empty tweet: 882627270321602560, tweet info: This is Stanley. He has his first swim lesson today. Doggle straps adjusted. Ready to go. 13/10 Phelps is nervous (IG: stanleythe_corgi) https://t.co/Nx52PGwH94
Empty tweet: 882268110199369728, tweet info: This is Alfy. You're witnessing his first watermelon experience. I think it was a success. 13/10 happy 4th Alfy 🇺🇸 https://t.co/fYP5RlutfA
Empty tweet: 882045870035918850, tweet info: This is Koko. Her owne

Empty tweet: 873697596434513921, tweet info: None
Empty tweet: 873697596434513921, tweet info: None
Empty tweet: 873697596434513921, tweet info: None
Empty tweet: 873697596434513921, tweet info: None
Empty tweet: 873697596434513921, tweet info: None
Empty tweet: 873697596434513921, tweet info: None
Empty tweet: 873580283840344065, tweet info: We usually don't rate Deck-bound Saskatoon Black Bears, but this one is h*ckin flawless. Sneaky tongue slip too. 13/10 would hug firmly https://t.co/mNuMH9400n
Empty tweet: 873337748698140672, tweet info: RT @dog_rates: This is Sierra. She's one precious pupper. Absolute 12/10. Been in and out of ICU her whole life. Help Sierra below

https:/…
Empty tweet: 873213775632977920, tweet info: This is Sierra. She's one precious pupper. Absolute 12/10. Been in and out of ICU her whole life. Help Sierra below

https://t.co/Xp01EU3qyD https://t.co/V5lkvrGLdQ
Empty tweet: 872967104147763200, tweet info: Here's a very large dog. He has a date later. Politely

Empty tweet: 865718153858494464, tweet info: Meet Boomer. He's just checking pup on you. Hopes you had a good day. If not, he hopes he made it better. 13/10 extremely good boy https://t.co/pozUoHLkGg
Empty tweet: 865359393868664832, tweet info: This is Sammy. Her tongue ejects without warning sometimes. It's a serious condition. Needs a hefty dose from a BlepiPen. 13/10 https://t.co/g20EmqK7vc
Empty tweet: 865006731092295680, tweet info: This is Nelly. He really hopes you like his Hawaiian shirt. He already tore the tags off. 13/10 h*ck of a puppurchase https://t.co/LbkG5CiM7o
Empty tweet: 864873206498414592, tweet info: We only rate dogs. Please don't send in Jesus. We're trying to remain professional and legitimate. Thank you... 14/10 https://t.co/wr3xsjeCIR
Empty tweet: 864279568663928832, tweet info: This is Meatball. He doing what's known in the industry as a mid-strut mlem. H*ckin fancy boy. 12/10 I'd do anything for Meatball https://t.co/S2HdmFFPck
Empty tweet: 86419739836464742

Empty tweet: 856330835276025856, tweet info: None
Empty tweet: 856330835276025856, tweet info: None
Empty tweet: 856330835276025856, tweet info: None
Empty tweet: 856330835276025856, tweet info: None
Empty tweet: 856330835276025856, tweet info: None
Empty tweet: 856330835276025856, tweet info: None
Empty tweet: 856282028240666624, tweet info: This is Cermet, Paesh, and Morple. They are absolute h*ckin superstars. Watered every day so they can grow. 14/10 for all https://t.co/GUefqUmZv8
Empty tweet: 855857698524602368, tweet info: HE'S LIKE "WAIT A MINUTE I'M AN ANIMAL THIS IS AMAZING HI HUMAN I LOVE YOU AS WELL" 13/10 https://t.co/sb73bV5Y7S
Empty tweet: 855851453814013952, tweet info: Here's a puppo participating in the #ScienceMarch. Cleverly disguising her own doggo agenda. 13/10 would keep the planet habitable for https://t.co/cMhq16isel
Empty tweet: 855818117272018944, tweet info: I HEARD HE TIED HIS OWN BOWTIE MARK AND HE JUST WANTS TO SAY HI AND MAYBE A NOGGIN PAT SHOW SOME RESP

...

Empty tweet: 670704688707301377, tweet info: Meet Danny. He's too good to look at the road when he's driving. Absolute menace. 6/10 completely irresponsible https://t.co/I1lMUy1FqH
Empty tweet: 670691627984359425, tweet info: This is Ester. He has a cocaine problem. This is an intervention Ester. We all care about you. 8/10 https://t.co/eCVj2xT59V
Empty tweet: 670679630144274432, tweet info: This is Pluto. He's holding little waddling dog hostage. Little waddling dog very desperate at this point sos. 8/10 https://t.co/HMcD9SLOAN
Empty tweet: 670676092097810432, tweet info: This is Bloo. He's a Westminster Cîroc. Doesn't think Bart deserves legs. Nice flowers. 8/10 https://t.co/IAc1QCczMc
Empty tweet: 670668383499735048, tweet info: This is Phineas. He's a magical dog. Only appears through the hole of a donut. 10/10 mysterious pup https://t.co/NECxEHN5YU
Empty tweet: 670474236058800128, tweet info: Honor to rate this dog. Great teeth. Nice horns. Unbelievable posture. Fun to pet. Big en

Empty tweet: 669683899023405056, tweet info: This is Kloey. Her mother was a unicorn. 10/10 https://t.co/NvKJRYDosA
Empty tweet: 669682095984410625, tweet info: Meet Louie. He just pounded that bottle of wine. 9/10 goodnight Louie https://t.co/RAwZvMKRZB
Empty tweet: 669680153564442624, tweet info: This is Shawwn. He's a Turkish Gangrene Robitussin. Spectacular tongue. Cranks out push-ups. 8/10 #NoDaysOff #swole https://t.co/IQFZKNUlXx
Empty tweet: 669661792646373376, tweet info: This is a brave dog. Excellent free climber. Trying to get closer to God. Not very loyal though. Doesn't bark. 5/10 https://t.co/ODnILTr4QM
Empty tweet: 669625907762618368, tweet info: This is Penny. She's having fun AND being safe. 12/10 very responsible pup https://t.co/eqeWw67oU7
Empty tweet: 669603084620980224, tweet info: Very human-like. Cute overbite smile *finger to earpiece* I'm being told that the dog is actually on the right
10/10 https://t.co/MSIbWu4YYs
Empty tweet: 669597912108789760, tweet info: 

Empty tweet: 668655139528511488, tweet info: Say hello to Winifred. He is a Papyrus Hydrangea mix. Can tie shoes. 11/10 inspiring pup https://t.co/mwnBN6ZkPt
Empty tweet: 668645506898350081, tweet info: Incredibly rare dog here. Good at bipedalism. Rad blue spikes. Ready to dance. 11/10 https://t.co/70X1TIXn38
Empty tweet: 668643542311546881, tweet info: Fascinating dog here. Loves beach. Oddly long nose for dog. Massive ass paws. Hard to cuddle w. 3/10 would still pet https://t.co/IiSdmhkC5N
Empty tweet: 668641109086707712, tweet info: Meet Hanz. He heard some thunder. 10/10 https://t.co/pKJK23j0QZ
Empty tweet: 668636665813057536, tweet info: This is an Irish Rigatoni terrier named Berta. Completely made of rope. No eyes. Quite large. Loves to dance. 10/10 https://t.co/EM5fDykrJg
Empty tweet: 668633411083464705, tweet info: This is Churlie. He likes bagels. 10/10 https://t.co/k8P6FZlzAG
Empty tweet: 668631377374486528, tweet info: Meet Zeek. He is a grey Cumulonimbus. Zeek is hungry. 

Empty tweet: 667861340749471744, tweet info: This is a Shotokon Macadamia mix named Cheryl. Sophisticated af. Looks like a disappointed librarian. Shh (lol) 9/10 https://t.co/J4GnJ5Swba
Empty tweet: 667832474953625600, tweet info: THE EYES 12/10

I'm sorry. These are supposed to be funny but your dogs are too adorable https://t.co/z1xPTgVLc7
Empty tweet: 667806454573760512, tweet info: This is Filup. He is overcome with joy after finally meeting his father. 10/10 https://t.co/TBmDJXJB75
Empty tweet: 667801013445750784, tweet info: OMIGOD 12/10 https://t.co/SVMF4Frf1w
Empty tweet: 667793409583771648, tweet info: Dogs only please. Small cows and other non canines will not be tolerated. Sick tattoos tho 8/10 https://t.co/s1z7mX4c9O
Empty tweet: 667782464991965184, tweet info: Super rare dog. Endangered (?). Thinks it's funny. Mocks everything I say. Colorful af. Has wings (dope). 9/10 https://t.co/BY8nQAMz0x
Empty tweet: 667773195014021121, tweet info: This is a rare Hungarian Pinot named

Empty tweet: 667065535570550784, tweet info: Here we have a Hufflepuff. Loves vest. Eyes wide af. Flaccid tail. Matches carpet. Always a little blurry. 8/10 https://t.co/7JdgVqDnvR
Empty tweet: 667062181243039745, tweet info: This is Keet. He is a Floridian Amukamara. Absolutely epic propeller hat. Pristine tongue. Nice plaid. 10/10 https://t.co/tz1lpuvXLA
Empty tweet: 667044094246576128, tweet info: 12/10 gimme now https://t.co/QZAnwgnOMB
Empty tweet: 667012601033924608, tweet info: This is Klevin. He laughs a lot. Very cool dog. 9/10 https://t.co/bATAbPNv0m
Empty tweet: 666996132027977728, tweet info: This is Carll. He wants to be a donkey. But also a soccer star. Dreams big. 10/10 https://t.co/SVpNbhaIMk
Empty tweet: 666983947667116034, tweet info: This is a curly Ticonderoga named Pepe. No feet. Loves to jet ski. 11/10 would hug until forever https://t.co/cyDfaK8NBc
Empty tweet: 666837028449972224, tweet info: My goodness. Very rare dog here. Large. Tail dangerous. Kinda fat. Only 

In [ ]:
public_metrics_list

In [ ]:
 df_public_metrics = pd.DataFrame(public_metrics_list)

In [ ]:
df_public_metrics.head(10)

In [ ]:
df_public_metrics.loc[:,"public_metrics"]

In [ ]:
df_public_metrics.head(10)


In [ ]:
# rows were public_metrics are NaN
df_public_metrics[df_public_metrics['public_metrics'].isna()]


In [ ]:
from pandas import json_normalize
# select non-NaN rows (where ~ is not )
mask=~df_public_metrics['public_metrics'].isna()
# unnest non-nan values
temp=json_normalize(df_public_metrics.loc[mask,'public_metrics'])
# join the result with the original
df_public_metrics = df_public_metrics.join(temp)

In [ ]:
df_public_metrics.drop('public_metrics', axis=1, inplace=True)

In [ ]:
df_public_metrics

In [ ]:
df_public_metrics.to_csv('tweet_json.txt', index=False)

In [ ]:
dfj = pd.read_csv('tweet_json.txt')

## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



In [ ]:
df.head(10)

In [ ]:
image_data.head(10)

In [ ]:
dfj.head(10)

In [ ]:
df.info()

In [ ]:
image_data.info()

In [ ]:
dfj.info

In [ ]:
df.isnull().sum()

In [ ]:
dfj.isnull().sum()

In [ ]:
image_data.isnull().sum()

In [ ]:
sum(df.duplicated())




In [ ]:
sum(dfj.duplicated())

In [ ]:
sum(image_data.duplicated())

In [ ]:
image_data.describe()

In [ ]:
df.describe()

In [ ]:
df_public_metrics.describe()

In [ ]:
df.nunique()

In [ ]:
dfj.nunique()

In [ ]:
image_data.nunique()

In [ ]:
np.sort(df['rating_numerator'].unique()) 

In [ ]:
np.sort(df['rating_denominator'].unique())

In [ ]:
df['rating_denominator'].value_counts()

In [ ]:
df['rating_numerator'].value_counts()

### Quality issues
1. Remove doggo, floofer, pupper, puppo

2. Wrong Datatype img_num Column should be in string

3. Wrong Datatype Name Column should be in string

4. Change tweet_id from an integer to a string

5. Timestamp is not of datetime format

6. Correct column contain some invalid name

7. Delete retweets .

8. Checking for duplicates and deleting them.

9.  Filling the fields which has no values

### Tidiness issues
1. Merge the dataframe twitter_archive, dataframe image_predictions, and tweet_json dataframes .
2. doggo, floofer, pupper, puppo these 4 variables shoule be combined into one  variable Dog Type.
3. Remove columns with too many missing values.
* retweeted_status_user_id
* retweeted_status_id
* retweeted_status_timestamp
* in_reply_to_user_id
* in_reply_to_status_id



## Create a copy of the all dataset for further  cleaning

In [ ]:
twitter = df.copy()
image = image_data.copy()
json = dfj.copy()

In [ ]:
json.drop(['text'], axis=1, inplace= True) 

In [ ]:
twitter.drop(['source'], axis=1, inplace= True) 

# Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

## Tidiness Problem 1

For rectifying the tidiness issue we need to merge the df twitter archive, df image_predictions , df tweet_json using python CONCAT command



In [ ]:
df2 = pd.concat([twitter, image, json], join='outer', axis=1)

In [ ]:
df2.head()


In [ ]:
df2.columns

In [ ]:
df2.shape

### Tidiness Problem 2

#### Define: doggo, floofer, pupper, puppo these 4 variables shoule be combined into one variable Dog Type.

We can use the .EXTRACT() method for that. But because of having multiple dog stages in a row, we need to consider that an write the code.

#### Code

In [ ]:
df2.groupby(["doggo", "floofer", "pupper", "puppo"]).size().reset_index().rename(columns={0: "count"})

In [ ]:
df2.doggo.replace('None', '', inplace=True) 
df2.doggo.replace(np.NaN, '', inplace=True)
df2.floofer.replace('None', '', inplace=True) 
df2.floofer.replace(np.NaN, '', inplace=True)
df2.pupper.replace('None', '', inplace=True) 
df2.pupper.replace(np.NaN, '', inplace=True)
df2.puppo.replace('None', '', inplace=True) 
df2.puppo.replace(np.NaN, '', inplace=True);

In [ ]:
df2['dogtype'] = df2.doggo + df2.floofer + df2.pupper + df2.puppo
df2.loc[df2.dogtype == 'doggopupper', 'dog_type'] = 'doggo, pupper'
df2.loc[df2.dogtype == 'doggopuppo', 'dog_type'] = 'doggo, puppo'
df2.loc[df2.dogtype == 'doggofloofer', 'dog_type'] = 'doggo, floofer'

In [ ]:
df2.dogtype.value_counts()

In [ ]:
df2.dogtype.unique()

In [ ]:
for idx, row in df2.iterrows():
    if ('doggoNoneNoneNone' in row['dogtype']):
        df2.loc[idx, 'dog_type'] = 'doggo'
    elif ('NoneNoneNonepuppo' in row['dogtype']):
        df2.loc[idx, 'dog_type'] = 'puppo'
    elif ('NoneNonepupperNone' in row['dogtype']):
        df2.loc[idx, 'dog_type'] = 'pupper'
    elif ('NoneflooferNoneNone' in row['dogtype']):
        df2.loc[idx, 'dog_type'] = 'floofer'
    elif ('NoneNoneNoneNone' in row['dogtype']):
        df2.loc[idx, 'dog_type'] = None
    else:
        df2.loc[idx, 'dog_type'] = 'multiple'
df2.head()  

In [ ]:
df2= df2.drop(['dogtype'] , axis =1)

#### Test

We can now look at column that we created.

In [ ]:
df2.dog_type.value_counts()

Now we can use the .DROP() method to remove doggo,floofer etc... those unwanted names

In [ ]:
df2.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)

Now we can see for sure

In [ ]:
df2.head()

### Tidiness Problem 3

#### Define - Remove columns with too many missing values

#### Code - We can use the .drop() method in here as well

In [ ]:
df2.drop(['in_reply_to_status_id', 'in_reply_to_user_id'], axis=1, inplace=True)
df2.head()

# Addressing the Quality Problem

## Quality Issue 1 - Change tweet_id from an integer to a string .

In [ ]:
df2['tweet_id'] = df2['tweet_id'].astype(str)

Test

In [ ]:
type(df2.iloc[0,0])

## Quality Issue 2 - Timestamp is not of datetime format

Creating the Timestamp in "Year-Month-Date" Form

In [ ]:
 #Change the 'timestamp' column to a datetime object
df2['timestamp'] = pd.to_datetime(df2['timestamp'], format = "%Y-%m-%d ")

In [ ]:
df2.head()

## Quality Issue 3- Delete Retweets

In [ ]:
df2 = df2[df2.retweeted_status_id.isnull()]
df2 = df2[df2.retweeted_status_user_id.isnull()]
df2 = df2[df2.retweeted_status_timestamp.isnull()]

In [ ]:
df2 = df2.drop(['retweeted_status_id',"retweeted_status_id","retweeted_status_user_id","retweeted_status_timestamp" ] , axis=1)

Test

We can just check the column names.

In [ ]:
df2.columns

## Quality Issue 4 -  Cleaning the name column. ( Replacing invalid names like  'a','an' , 'not', 'quite', etc.. with None)


In [ ]:
df2['name'] = df2.name.replace(['None', 'a', 'an', 'very','the', 'not', 'quite', 'actually'], 'None' , inplace = True);

Test

In [ ]:
df2[df2['name']=='a']

## Quality Issue 5 - Wrong Data Type in Source Column- Source Column should be in Category DataType

In [ ]:
df2['source'] = df2['source'].astype('category')

Test - We can check the results using type method

In [ ]:
df['source'].dtypes


## Quality Issue 6- Wrong Data Type in img_num - It should be String

In [ ]:
df2['img_num'] = df2['img_num'].astype(str)

Test

In [ ]:
df2.columns.get_loc("img_num")

In [ ]:
type(df2.iloc[0,9])

## Quality Issue 7 -  Remove columns no longer needed

In [ ]:
df2.columns

In [ ]:
df2.head()

In [ ]:
df2.drop(['img_num','id','name'], axis = 1, inplace=True)

Test

In [ ]:
df2.info()


## Quality Issue 8 - Checking for Duplicates and Deleting them

In [ ]:
df2.drop_duplicates(inplace = True)

Test

In [ ]:
df2[df2.duplicated()].sum()

In [ ]:
df2.info()

## Quality Issue 9 - Filling the fields which has no values

In [ ]:
df2['jpg_url'] = df2['jpg_url'].fillna("None")
df2['p1'] = df2['p1'].fillna("None")
df2['p1_conf'] = df2['p1_conf'].fillna(0)
df2['p1_dog'] = df2['p1_dog'].fillna("None")
df2['p2'] = df2['p2'].fillna("None")
df2['p2_conf'] = df2['p2_conf'].fillna(0)
df2['p2_dog'] = df2['p2_dog'].fillna("None")
df2['p3'] = df2['p3'].fillna("None")
df2['p3_conf'] = df2['p3_conf'].fillna(0)
df2['p3_dog'] = df2['p3_conf'].fillna("None")
df2['lang'] = df2['lang'].fillna("None")
df2['retweet_count'] = df2['retweet_count'].fillna(0)
df2['reply_count'] = df2['reply_count'].fillna(0)
df2['like_count'] = df2['like_count'].fillna(0)
df2['quote_count'] = df2['quote_count'].fillna(0)
df2['dog_type'] = df2['dog_type'].fillna("None")
df2['expanded_urls'] = df2['expanded_urls'].fillna("None")









In [ ]:
df2.info()

### Storing our cleaned data into CSV File

In [ ]:
df2.to_csv('twitter_archive_master.csv', encoding='utf-8')

 ## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

Import the cleaned Dataset 

In [ ]:
 df3 = pd.read_csv('twitter_archive_master.csv')

In [ ]:
df3.head()

In [ ]:
df3.shape

## Insight 1
### Plot between Count of Retweet and Count of Likes :

In [ ]:
plt.figure(figsize = [10,5])
bins = np.arange(df3['like_count'].min() , df3['like_count'].max() + 500, 500)
df3.like_count.plot(kind='hist',  bins=bins )
bins = np.arange(df3['retweet_count'].min() , df3['retweet_count'].max() + 500, 500)
df3.retweet_count.plot(kind='hist', bins=bins )

plt.xlabel('count')
plt.legend()
plt.xlim(200,30000)
plt.ylim(0,300)
plt.title( 'like/retweet');

Observation :
it is clearly shown that the count of likes are more than the count of retweet .

## Insight 2
### Finding out the most Popular Dog Breed

In [ ]:
df3.dog_type.value_counts()


In [ ]:
dog_breed = df3.groupby('dog_type').filter(lambda x: len(x) >= 25)

dog_breed['dog_type'].value_counts().plot(kind = 'barh')
plt.title('Most Rated Dog Breed')
plt.xlabel('Count')
plt.ylabel('Breed of dog');

Observation - We can see that most of the dogs do not have a breed. Out of the dogs who has a breed Pupper is the most frequent dog breed followed by doggo.

## Insight 3
### Plot of the Languages.¶

In [ ]:
g = df3['lang'].value_counts()
g.plot(kind='bar')
plt.xlabel('Language')
plt.ylabel('Count')
plt.title('Count of Languages.');

Observation - As you can see English is the most used language

## Insight 4 and Visualization
### Checking whether the highest retweets do recieve the most replys

In [ ]:
df3.plot(x='retweet_count', y='reply_count', kind='scatter')
plt.xlabel('Retweet Counts')
plt.ylabel('Replys')
plt.title('Retweet Counts by Replys Scatter Plot')

fig = plt.gcf()
fig.savefig('output3.png',bbox_inches='tight');

As you can see, higher the retweet count higher the number of replys get.